## Решение краевой задачи

In [92]:
import numpy as np
import pandas as pd

### Получение системы уравнений

In [226]:
class Lab3(object):
    def __init__(self, start_condition_function, x_equation, y_equation, start_x, end_x, step, second_equation_value, end_y):
        self._x_values = self.get_x_values(start_x, end_x, step)
        self._end_x = end_x
        self._start_condition_function = start_condition_function
        self._step = step
        self._x_equation = x_equation
        self._y_eqution = y_equation
        self._second_equation_value = second_equation_value
        self._end_y = end_y
    
    def get_x_values(self, start_x, end_x, step):
        return np.arange(start_x, end_x, step)
    
    def _get_left_first_derivative_coefficients(self):
        return np.array([-1./self._step, 1./self._step])
    
    def get_first_equation(self):
        return self._start_condition_function(self._get_left_first_derivative_coefficients)
    
    def _get_mid_first_equation(self):
        return np.array([-1./(2.*self._step), 0, 1./(2.*self._step)])
    
    def _get_second_derivative_coefficients(self):
        return np.array([1./self._step**2, -2./self._step**2, 1./self._step**2])
    
    def _get_y_coefficients(self):
        return np.array([0., 1., 0.])
    
    def get_equation(self, x_value):
        second_derivative_coefficients = self._get_second_derivative_coefficients()
        first_derivative_value = self._x_equation(x_value) * self._get_mid_first_equation()
        y_equation_coefficients = self._y_eqution(x_value) * self._get_y_coefficients()
#         print("----------------------")
#         print(second_derivative_coefficients)
#         print(first_derivative_value)
#         print(y_equation_coefficients)
#         print("----------------------")
        return second_derivative_coefficients + first_derivative_value + y_equation_coefficients, self._second_equation_value
    
    def get_second_equation(self):
        return self.get_equation(self._x_values[1])
    
    def get_third_equation(self):
        coefficients, value = self.get_equation(self._x_values[2])
        value -= coefficients[2]*self._end_y
        return coefficients[:-1], value
    
    @staticmethod
    def solve(a,b,c,f):
        a, b, c, f = map(lambda k_list: map(float, k_list), (a, b, c, f))
        a = list(a)
        b = list(b)
        c = list(c)
        f = list(f)
        alpha = [0]
        beta = [0]
        n = len(f)
        x = [0]*n

        for i in range(n-1):
            alpha.append(-b[i]/(a[i]*alpha[i] + c[i]))
            beta.append((f[i] - a[i]*beta[i])/(a[i]*alpha[i] + c[i]))

        x[n-1] = (f[n-1] - a[n-2]*beta[n-1])/(c[n-1] + a[n-2]*alpha[n-1])

        for i in reversed(range(n-1)):
            x[i] = alpha[i+1]*x[i+1] + beta[i+1]

        return x
    
    @staticmethod
    def check(answer, system, f):
        values_test = np.sum(system*anwer, axis=1)
        print("Проверка значений полученных после решения {}".format(values_test))
        print("Реальные значения системы {}".format(f))
    
    def get_system_of_equation(self):
        values = []
        coefficients, value = self.get_first_equation()
        coefficients_first = np.zeros(3)
        coefficients_first[[0,1]] = coefficients
        values.append(value)
        print("{:.1f}Y0 + {:.1f}Y1 = {:.1f}".format(coefficients[0], coefficients[1], value))
        coefficients, value = self.get_second_equation()
        values.append(value)
        print("{:.1f}Y0 {:.2f}Y1 + {:.1f}Y2 = {:.1f}".format(coefficients[0], coefficients[1], coefficients[2], value))
        coefficients_second = coefficients
        coefficients, value = self.get_third_equation()
        print("{:.1f}Y0 {:.2f}Y1 = {:.1f}".format(coefficients[0], coefficients[1], value))
        coefficients_third = np.zeros(3)
        coefficients_third[[1,2]] = coefficients
        values.append(value)
        return np.array([coefficients_first, coefficients_second, coefficients_third, values])
    
    def main(self):
        system = self.get_system_of_equation()
        f = system[-1, :]
        system = system[:-1]
        a = np.diagonal(system, offset=-1)
        b = np.diagonal(system, offset=1)
        c = np.diagonal(system, offset=0)
        anwer = self.solve(a,b,c,f)
#         anwer = [2.2675624378288775, 2.2041843159374337, 2.1666499479238346]
        print("Решение системы {}".format(anwer))
        self.check(anwer, system, f)

In [227]:
def start_condition(derivative):
    return 2*derivative() + np.array([1., 0.]), 1.

def x_equation(x):
    return x

def y_equation(x):
    return -0.5/x

start_x = 2.
end_x = 2.3
end_y = 2.15
function_value = 1.
step = 0.1
test = Lab3(start_condition, x_equation, y_equation, start_x, end_x, step, function_value, end_y)
test.main()

-19.0Y0 + 20.0Y1 = 1.0
89.5Y0 -200.24Y1 + 110.5Y2 = 1.0
89.0Y0 -200.23Y1 = -237.6
Решение системы [2.2467837522582905, 2.1844445646453763, 2.157875699790158]
Проверка значений полученных после решения [   1.            2.12339188 -237.65      ]
Реальные значения системы [   1.      1.   -237.65]
